In [1]:
from src.api import API
from src.dataloader import DataLoader
from src.prompter import prompter_factory
from src.evaluator import evaluate_model

In [2]:
TOGETHER_API_KEY = "92a6ac4a8feb39c91b4a3f77219e9c452d927f5f4d543d5969cc11c210795719"
BASE_URL = "https://api.together.xyz"
ALLOWED_SYMPTOMS = ['anxiety', 'concentration problems', 'constipation', 'cough',
                    'diarrhea', 'fatigue', 'fever', 'headache', 'nausea', 
                    'numbness and tingling', 'pain', 'poor appetite', 'rash', 
                    'shortness of breath', 'trouble drinking fluids', 'vomiting', 'other']

In [3]:
api = API(api_key=TOGETHER_API_KEY, base_url=BASE_URL)
client = api.get_openai()

In [4]:
dataloader = DataLoader(path="data/")
print(dataloader.list_csv_files())

['data/batch_1_gs.csv', 'data/batch_2_gs.csv', 'data/batch_3_gs.csv', 'data/batch_4_gs.csv', 'data/batch_5_gs.csv', 'data/batch_6_gs.csv', 'data/batch_7_gs.csv', 'data/batch_8_gs.csv', 'data/batch_9_gs.csv', 'data/batch_10_gs.csv']


In [5]:
df = dataloader.get_standardized_dataframe(context_col="Text Data",
                                           target_binary_col="symptom_status_gs",
                                           target_multilabel_col="symptom_detail_gs",
                                           keep_other_cols=True)

In [50]:
prompter = prompter_factory(prompter_type="binary",
                            client=client,
                            model="meta-llama/Llama-2-70b-chat-hf")

In [51]:
prompt = "Are any medical symptoms mentioned in the transcript"
context = "i have a headache every time i see a cat and i hear voices that are not there"
print(prompter.generate_single(prompt=prompt, context=context))

nan


In [75]:
prompt = "<s>[INST] The following question requires you to output a single word. Your answer will have to be Yes or No. It is very important that you follow this instruction. Are any medical symptoms mentioned in the transcript? i have a headache every time i see a cat and i hear voices that are not there [/INST]"

In [87]:
input = f"""
        <<SYS>>
        Analyze the text in the content and evaluate if there are any medical symptoms mentioned in the transcript for 'User'.\n
        And return your medical analysis only as : Yes or No. It is of the uttermost importance that your response is either Yes or No.\n
        <</SYS>>
        [INST]
        User:{context}
        [/INST]\n

        Assistant:
    """

In [91]:
completion = client.completions.create(
    model="meta-llama/Llama-2-70b-hf",
    temperature=0,
    prompt=input,
    max_tokens=100,
)
print(completion.choices[0])

CompletionChoice(finish_reason='length', index=0, logprobs=None, text="\n        <SYS>\n        Analyze the text in the content and evaluate if there are any medical symptoms mentioned in the transcript for 'User'.\n\n        And return your medical analysis only as : Yes or No. It is of the uttermost importance that your response is either Yes or No.\n\n        <</SYS>\n        [INST]\n        User:i have a headache every time i see a cat and i hear voices that are not")


In [46]:
model = "meta-llama/Llama-2-70b-chat-hf"

prompt = """To change the brakes on your car, you start by"""

output = client.completions.create(
  prompt = prompt, 
  model = model, 
  max_tokens = 64,
  temperature = 0,
  top_p = 0.7,
  #stop = [] # add any sequence you want to stop generating at. 
)

# print generated text
print(output.choices[0].text)

 removing the wheel and caliper. Then, you remove the brake pads and any other components that need to be replaced. Next, you install the new brake pads and reassemble the brake caliper. Finally, you reinstall the wheel and test the brakes to make sure they
